In [1]:
import os
import re
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
from datetime import timedelta
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt

from konlpy.tag import Okt
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.models import word2vec
from sklearn.manifold import TSNE

In [2]:
data_dic = {}
idx = 0

for curDir, dirs, files in tqdm(os.walk("../4.프로젝트/NewsData")):
    for f in files:
        try:
            path = os.path.join(curDir, f)
            data = pd.read_csv(path, encoding='cp949')
            data_dic[idx] = data
        except:
            path = os.path.join(curDir, f)
            print(f'========={path} ERROR=========')
        idx += 1

189it [00:34,  5.47it/s]


In [3]:
df = pd.concat(data_dic).reset_index(drop=True)
df['Date'] = df['Date'].apply(lambda x: x.split('.')[0] + '-' + x.split('.')[1] + '-' + x.split('.')[2])
df['Date'] = pd.to_datetime(df['Date'])

df['Title_Article'] = df['Title'] + df['Article']
df = df.drop(['Title', 'Article'], axis=1)

In [ ]:
def preprocessing(data, okt, remove_stopwords=False, stop_words=[]):
    data_text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]', '', data)
#     word_data = okt.morphs(data_text, stem=True)   # stem, 어간 추출 (해야지 -> 하다)로 변환
#     word_data = okt.phrases(data_text)    
    word_data = okt.nouns(data_text)
    
    if remove_stopwords:
        word_data = [token for token in word_data if not token in stop_words and len(token) != 1]
    
    return word_data

In [5]:
okt = Okt()
clean_data = []

for text in tqdm(df['Title']):
    clean_data.append(preprocessing(text, okt))

100%|██████████| 783135/783135 [24:56<00:00, 523.43it/s]


In [9]:
df['Title_Token'] = clean_data
df = df.drop(['Title', 'Article'], axis=1)

In [13]:
df.to_csv('./Data/KonlpyData.csv', index=False)